




<p style='text-align:center; font-size: 30px;'>ELAN to CSV for multiple files at once</p>

<br>
<br>

for each ELAN file in the folder, there will be a csv file created with the same name
 
<br>
<br>

<div class="warning" style='padding:0.1em; background-color: #FDAE44; color:#51247a; border-style: solid; border-color: #CC5500 '>
<span>
<p style='margin-top:1em; text-align:center'>
<b>Never use this script on your main files. always use it on a copy of your files! </b> 
<br>
</p>
<p style='margin-left:1em;'></p></span>
</div>
 

In [1]:
import os
import re
import csv
import glob
from bs4 import BeautifulSoup as bs

dir_path = 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan/'

print (dir_path)
files = glob.glob(dir_path + "*.eaf")
print(files)
print ("+++ FILES LOADED: ", len(files))

C:/Users/barth/Documents/LDACA/AusESL/edited_elan/
['C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_002_Stella.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_006_Tanya.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_008_Kelsey.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_009_Joyce.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_011_Rosa.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_012_Ruby.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_022_Mei.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_024_Liu.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_029_Ning.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_036_Liling.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_037_Mingzhu.eaf', 'C:/Users/barth/Documents/LDACA/AusESL/edited_elan\\AusESL_MF_038_Xiulan.eaf', 'C:/Users/bar

itterate over all files and read the XML tags into a CSV file

A filter function removes all unwanted tiers

In [2]:
def time_value_to_hms(time_value):
    # trasforms the millisec int into hh:mm:ss:mmmm string
    millis= time_value
    millis = int(millis)
    time_value_padded =  "000" + time_value
    mill = int(time_value_padded[-3:])
    #print (mill)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)
    hours=(millis/(1000*60*60))%24
    
    if len(str(mill)) < 3:
        mill = str(mill) + "0"
    if len(str(mill)) < 3:
        mill = str(mill) + "0"
    if len(str(mill)) < 3:
        mill = str(mill) + "0"
    
    if len(str(minutes)) < 2:
        minutes = "0" + str(minutes)
    minutes = str(minutes)[:2]
        
    if len(str(seconds)) < 2:
        seconds = "0" + str(seconds)
    
    if len(str(hours)) < 2:
        hours = "0" + str(hours)
    hours = str(hours)[:1]
    
      
    timestring = str(hours) + ":" + str(minutes) + ":" + str(seconds) + "." + str(mill)
    print (timestring)
    return timestring

def time_slot_dict_maker(soup):
        # create the time slot reference dict
        timeSlotDict = {}
        time_slots = soup.findAll("time_slot")
        
        for time_slot in time_slots:
            time_value = time_value_to_hms(time_slot["time_value"])
            timeSlotDict[time_slot["time_slot_id"]] = time_value
        
        return timeSlotDict

def write_to_file(outfile, fileList):
    with open(outfile, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(fileList)

def alignable_annotation():
    
    annotation_id = annotation['annotation_id']
    time_slot_ref1 = annotation['time_slot_ref1']
    time_slot_ref2 = annotation['time_slot_ref2']
    text = annotation.annotation_value.get_text()

    annotationRefDict[annotation_id] = [time_slot_ref1, time_slot_ref2]

    return [filename[:-4],
            annotation_id.replace("a", ""),
            timeSlotDict[time_slot_ref1],
            timeSlotDict[time_slot_ref2],
            tier_id,
            text]

def ref_annotation():
    
    annotation_id = annotation['annotation_id']
    annotation_ref = annotation['annotation_ref']
    if 'prefious_annotation' in annotation:
        previous_annotation = annotation['previous_annotation']
    text = annotation.annotation_value.get_text()

    time_slot_ref1 = annotationRefDict[annotation_ref][0]
    time_slot_ref2 = annotationRefDict[annotation_ref][1]

    annotationRefDict[annotation_id] = [time_slot_ref1, time_slot_ref2]


    return [filename[:-4],
            annotation_id.replace("a", ""),
            timeSlotDict[time_slot_ref1],
            timeSlotDict[time_slot_ref2],
            tier_id,
            text]

def filter_list(fileList):
    # only keep the tiers you want
    filteredList = [headers]
    for each in fileList:
        speaker = each[4]
        if speaker.startswith("INT: "):
            filteredList.append(each)
        elif speaker.startswith("LaBB-CAT_PNT: "):
            filteredList.append(each)
    return filteredList
    
headers = ['Transcript', 
           'IU_number', 
           'start_time', 
           'end_time', 
           'speaker', 
           'IU']

outList = []

for file in files:

    filename = os.path.basename(file)
    outfile = dir_path + filename[:-4] +".csv"
    
    with open(file, "r", encoding="utf-8") as f:
        
        annotationRefDict = {}
        fileList = []
        fileList.append(headers)
        
        soup = bs(f, "lxml")
        
        timeSlotDict = time_slot_dict_maker(soup)

        tiers = soup.find_all('tier')
        for tier in tiers:
            
            #tier level data
            linguistic_type_ref = tier['linguistic_type_ref']
            tier_id = tier['tier_id']
            
            # annotation level data
            # there are two types of annotation tags:
            
            for annotation in tier.find_all('alignable_annotation'):
                tierList = alignable_annotation()
                fileList.append(tierList)
                
            for annotation in tier.find_all('ref_annotation'):
                tierList = ref_annotation()
                fileList.append(tierList)
    
    # only include the tiers you want
    filteredList = filter_list(fileList)
    
    head = filteredList[0]
    
    filteredList = filteredList[1:]
    
    sorted_list = sorted(filteredList, key=lambda x: x[2])
    finalList = []
    finalList.append(head)
    numi = 0
    for each in sorted_list:
        numi += 1
        each[1] = str(numi)
        finalList.append(each)
    
    
    write_to_file(outfile, finalList) 
    print(" +++ DONE +++")

C:\Users\barth\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


0:00:00.000
0:00:00.000
0:00:04.960
0:00:05.150
0:00:07.720
0:00:08.460
0:00:11.680
0:00:11.715
0:00:12.110
0:00:13.955
0:00:16.410
0:00:16.760
0:00:19.750
0:00:20.000
0:00:20.000
0:00:21.185
0:00:21.235
0:00:22.205
0:00:22.570
0:00:23.430
0:00:23.837
0:00:23.905
0:00:25.170
0:00:25.173
0:00:25.173
0:00:25.670
0:00:25.682
0:00:26.220
0:00:28.250
0:00:28.360
0:00:29.800
0:00:30.200
0:00:32.000
0:00:32.000
0:00:32.770
0:00:33.170
0:00:33.260
0:00:35.960
0:00:36.310
0:00:41.400
0:00:41.410
0:00:41.580
0:00:41.600
0:00:42.000
0:00:42.000
0:00:42.900
0:00:43.570
0:00:45.000
0:00:50.770
0:00:51.000
0:00:51.000
0:00:51.360
0:00:51.930
0:00:52.500
0:00:53.215
0:00:53.730
0:00:54.400
0:00:57.000
0:00:57.000
0:00:58.195
0:00:58.200
0:00:58.925
0:00:58.960
0:01:01.530
0:01:02.355
0:01:04.515
0:01:04.545
0:01:05.180
0:01:08.000
0:01:08.000
0:01:08.275
0:01:09.000
0:01:09.000
0:01:10.495
0:01:11.850
0:01:11.890
0:01:12.000
0:01:12.000
0:01:12.190
0:01:13.470
0:01:13.495
0:01:13.820
0:01:14.765
0:01

0:00:00.000
0:00:02.130
0:00:02.820
0:00:05.550
0:00:06.275
0:00:09.175
0:00:09.645
0:00:10.450
0:00:10.895
0:00:11.220
0:00:11.425
0:00:15.130
0:00:15.170
0:00:15.550
0:00:16.130
0:00:17.110
0:00:17.690
0:00:18.630
0:00:18.775
0:00:18.920
0:00:19.265
0:00:21.645
0:00:21.690
0:00:22.500
0:00:22.315
0:00:25.500
0:00:25.915
0:00:28.355
0:00:28.595
0:00:29.635
0:00:30.455
0:00:32.540
0:00:33.550
0:00:34.260
0:00:34.320
0:00:35.110
0:00:35.290
0:00:35.750
0:00:35.770
0:00:39.690
0:00:39.860
0:00:40.500
0:00:40.745
0:00:41.735
0:00:41.930
0:00:41.960
0:00:42.390
0:00:44.190
0:00:44.810
0:00:47.750
0:00:48.100
0:00:48.950
0:00:49.240
0:00:50.770
0:00:52.200
0:00:55.340
0:00:56.000
0:00:58.560
0:00:59.350
0:00:59.700
0:00:59.830
0:01:00.720
0:01:01.290
0:01:03.320
0:01:03.545
0:01:05.555
0:01:05.735
0:01:07.130
0:01:08.230
0:01:09.000
0:01:09.885
0:01:10.943
0:01:10.943
0:01:10.950
0:01:11.750
0:01:12.315
0:01:12.450
0:01:12.460
0:01:12.970
0:01:13.860
0:01:13.860
0:01:16.290
0:01:17.250
0:01

0:00:00.000
0:00:06.000
0:00:06.000
0:00:12.740
0:00:13.000
0:00:18.000
0:00:18.320
0:00:19.000
0:00:19.160
0:00:26.260
0:00:26.300
0:00:28.410
0:00:28.900
0:00:34.980
0:00:36.320
0:00:41.730
0:00:41.730
0:00:46.500
0:00:46.500
0:00:50.920
0:00:51.600
0:00:53.330
0:00:53.650
0:00:58.120
0:00:58.760
0:01:03.180
0:01:03.620
0:01:09.810
0:01:09.525
0:01:09.835
0:01:10.000
0:01:18.700
0:01:19.110
0:01:20.760
0:01:21.000
0:01:21.260
0:01:21.540
0:01:26.899
0:01:27.900
0:01:28.000
0:01:31.259
0:01:31.690
0:01:31.930
0:01:33.370
0:01:33.570
0:01:35.430
0:01:35.710
0:01:37.940
0:01:38.540
0:01:46.860
0:01:48.600
0:01:50.630
0:01:50.990
0:01:52.760
0:01:53.450
0:01:53.690
0:01:54.000
0:01:57.985
0:01:58.000
0:01:58.000
0:01:58.485
0:01:59.000
0:01:59.120
0:01:59.480
0:02:00.000
0:02:00.105
0:02:00.455
0:02:02.000
0:02:02.150
0:02:02.325
0:02:02.600
0:02:06.550
0:02:07.110
0:02:09.700
0:02:09.780
0:02:16.320
0:02:17.000
0:02:17.205
0:02:17.515
0:02:18.000
0:02:18.000
0:02:19.590
0:02:20.190
0:02

0:00:00.000
0:00:06.000
0:00:08.000
0:00:10.000
0:00:10.000
0:00:11.000
0:00:11.000
0:00:11.000
0:00:14.000
0:00:14.000
0:00:14.000
0:00:14.433
0:00:15.000
0:00:15.000
0:00:15.513
0:00:16.195
0:00:19.000
0:00:19.000
0:00:19.605
0:00:20.000
0:00:26.000
0:00:26.459
0:00:26.500
0:00:26.500
0:00:28.000
0:00:28.000
0:00:29.000
0:00:29.000
0:00:31.278
0:00:32.240
0:00:34.000
0:00:34.000
0:00:35.500
0:00:35.500
0:00:41.500
0:00:41.500
0:00:42.220
0:00:42.790
0:00:46.327
0:00:47.000
0:00:48.000
0:00:48.000
0:00:49.000
0:00:49.000
0:00:52.000
0:00:52.000
0:00:53.000
0:00:53.000
0:00:54.000
0:00:54.000
0:00:55.802
0:00:55.802
0:00:57.000
0:00:57.000
0:00:58.000
0:00:58.000
0:01:04.000
0:01:04.000
0:01:05.000
0:01:05.000
0:01:05.700
0:01:06.540
0:01:08.000
0:01:08.000
0:01:13.000
0:01:13.000
0:01:17.000
0:01:17.000
0:01:17.000
0:01:17.000
0:01:18.000
0:01:18.000
0:01:19.000
0:01:19.000
0:01:24.000
0:01:24.000
0:01:24.710
0:01:24.710
0:01:26.384
0:01:27.491
0:01:29.000
0:01:29.000
0:01:33.601
0:01

...

0:00:00.000
0:00:09.000
0:00:09.725
0:00:11.000
0:00:11.000
0:00:14.000
0:00:14.000
0:00:14.000
0:00:14.785
0:00:15.775
0:00:16.000
0:00:16.000
0:00:22.000
0:00:22.000
0:00:23.695
0:00:24.212
0:00:26.142
0:00:26.695
0:00:30.000
0:00:30.000
0:00:36.625
0:00:37.457
0:00:37.500
0:00:37.500
0:00:44.270
0:00:44.638
0:00:45.000
0:00:45.000
0:00:52.148
0:00:52.200
0:00:52.200
0:00:53.700
0:00:59.500
0:00:59.500
0:01:03.000
0:01:03.270
0:01:06.700
0:01:06.700
0:01:09.000
0:01:09.455
0:01:14.000
0:01:14.000
0:01:16.845
0:01:17.277
0:01:18.000
0:01:18.000
0:01:23.000
0:01:23.000
0:01:23.767
0:01:24.168
0:01:30.948
0:01:31.000
0:01:31.000
0:01:31.000
0:01:34.000
0:01:34.805
0:01:36.000
0:01:36.000
0:01:39.945
0:01:40.250
0:01:41.000
0:01:41.000
0:01:44.150
0:01:45.195
0:01:46.000
0:01:46.000
0:01:51.000
0:01:51.000
0:01:51.895
0:01:52.000
0:01:57.000
0:01:57.000
0:02:00.000
0:02:00.245
0:02:01.000
0:02:01.000
0:02:03.975
0:02:04.560
0:02:06.000
0:02:06.000
0:02:10.300
0:02:11.145
0:02:11.700
0:02

0:00:00.000
0:00:08.000
0:00:08.770
0:00:10.700
0:00:14.160
0:00:14.665
0:00:18.000
0:00:26.415
0:00:27.520
0:00:32.412
0:00:32.835
0:00:36.905
0:00:37.655
0:00:38.895
0:00:39.210
0:00:40.830
0:00:42.000
0:00:43.000
0:00:43.245
0:00:48.815
0:00:49.925
0:00:56.955
0:00:57.892
0:01:01.302
0:01:02.145
0:01:07.935
0:01:08.000
0:01:12.000
0:01:12.625
0:01:17.185
0:01:17.755
0:01:25.555
0:01:25.955
0:01:28.435
0:01:29.320
0:01:34.700
0:01:34.630
0:01:38.310
0:01:38.505
0:01:41.945
0:01:42.000
0:01:46.400
0:01:46.895
0:01:47.000
0:01:47.655
0:01:48.000
0:01:49.000
0:01:56.755
0:01:57.000
0:02:02.850
0:02:02.515
0:02:07.445
0:02:07.897
0:02:18.800
0:02:18.317
0:02:25.857
0:02:27.140
0:02:37.870
0:02:38.420
0:02:46.270
0:02:47.210
0:02:52.530
0:02:53.385
0:02:57.445
0:02:57.695
0:03:01.345
0:03:01.975
0:03:02.000
0:03:02.455
0:03:03.550
0:03:03.710
0:03:05.000
0:03:05.530
0:03:10.730
0:03:11.515
0:03:13.000
0:03:13.215
0:03:15.000
0:03:15.000
0:03:15.220
0:03:15.500
0:03:16.575
0:03:17.000
0:03

0:00:00.000
0:00:04.000
0:00:04.650
0:00:07.850
0:00:08.730
0:00:17.450
0:00:18.200
0:00:32.240
0:00:32.755
0:00:40.925
0:00:40.960
0:00:46.180
0:00:46.467
0:00:51.537
0:00:51.855
0:01:02.915
0:01:03.402
0:01:06.462
0:01:07.125
0:01:12.405
0:01:13.500
0:01:17.405
0:01:18.165
0:01:26.215
0:01:26.425
0:01:33.195
0:01:33.315
0:01:42.925
0:01:43.687
0:01:45.467
0:01:46.275
0:01:55.245
0:01:56.000
0:01:56.545
0:01:56.965
0:01:57.000
0:01:57.215
0:02:00.215
0:02:00.405
0:02:05.845
0:02:06.595
0:02:12.705
0:02:13.545
0:02:18.465
0:02:18.805
0:02:20.765
0:02:20.910
0:02:26.000
0:02:26.685
0:02:28.745
0:02:29.485
0:02:40.735
0:02:41.232
0:02:44.932
0:02:46.131
0:02:52.961
0:02:53.345
0:02:57.215
0:02:57.750
0:02:59.560
0:03:00.120
0:03:12.000
0:03:12.400
0:03:12.415
0:03:12.725
0:03:13.000
0:03:13.495
0:03:15.405
0:03:15.480
0:03:18.490
0:03:19.710
0:03:25.500
0:03:26.615
0:03:30.145
0:03:30.705
0:03:39.765
0:03:40.150
0:03:43.930
0:03:45.170
0:03:48.170
0:03:48.687
0:03:49.917
0:03:50.287
0:03

0:00:00.000
0:00:02.000
0:00:03.000
0:00:03.000
0:00:04.000
0:00:04.000
0:00:05.000
0:00:05.000
0:00:06.000
0:00:06.000
0:00:07.770
0:00:07.770
0:00:08.000
0:00:11.000
0:00:11.000
0:00:13.490
0:00:13.490
0:00:14.415
0:00:18.000
0:00:18.000
0:00:19.000
0:00:19.000
0:00:21.645
0:00:22.000
0:00:22.000
0:00:22.897
0:00:22.897
0:00:27.000
0:00:27.000
0:00:29.870
0:00:29.870
0:00:30.308
0:00:32.000
0:00:32.000
0:00:35.000
0:00:35.000
0:00:35.800
0:00:35.849
0:00:36.000
0:00:36.000
0:00:40.639
0:00:41.695
0:00:42.500
0:00:42.500
0:00:48.725
0:00:49.000
0:00:49.000
0:00:49.277
0:00:49.887
0:00:50.877
0:00:55.500
0:00:55.500
0:00:56.787
0:00:56.948
0:01:00.438
0:01:00.540
0:01:02.000
0:01:02.000
0:01:02.180
0:01:03.795
0:01:04.000
0:01:04.000
0:01:06.000
0:01:06.000
0:01:09.000
0:01:09.000
0:01:09.795
0:01:11.412
0:01:12.762
0:01:14.145
0:01:15.000
0:01:15.000
0:01:21.000
0:01:21.000
0:01:26.815
0:01:28.265
0:01:29.500
0:01:29.500
0:01:30.765
0:01:31.765
0:01:38.000
0:01:38.000
0:01:38.650
0:01

0:00:00.000
0:00:03.000
0:00:03.980
0:00:06.000
0:00:06.290
0:00:08.750
0:00:09.100
0:00:17.180
0:00:18.170
0:00:21.180
0:00:23.000
0:00:24.000
0:00:24.980
0:00:35.430
0:00:35.960
0:00:40.250
0:00:40.870
0:00:41.800
0:00:42.862
0:00:49.942
0:00:50.602
0:01:02.222
0:01:03.910
0:01:10.865
0:01:10.870
0:01:19.790
0:01:20.805
0:01:25.525
0:01:27.262
0:01:33.572
0:01:33.926
0:01:42.686
0:01:43.350
0:01:53.590
0:01:53.685
0:01:57.355
0:01:57.857
0:02:02.187
0:02:02.810
0:02:06.260
0:02:06.780
0:02:20.445
0:02:21.220
0:02:33.420
0:02:34.150
0:02:38.450
0:02:39.325
0:02:52.475
0:02:53.700
0:03:08.700
0:03:09.150
0:03:14.120
0:03:14.940
0:03:20.500
0:03:20.925
0:03:30.895
0:03:31.970
0:03:38.137
0:03:38.328
0:03:45.648
0:03:46.334
0:03:52.714
0:03:54.110
0:04:08.100
0:04:08.150
0:04:21.300
0:04:22.000
0:04:33.250
0:04:34.820
0:04:49.500
0:04:51.488
0:05:00.520
0:05:00.520
0:05:05.000
0:05:05.470
0:05:08.700
0:05:09.520
0:05:11.290
0:05:12.500
0:05:14.365
0:05:14.720
0:05:19.820
0:05:23.810
0:05